### Import the dataset with labels

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pywt
import random
from scipy.io import loadmat

from tsfresh import extract_relevant_features

import csv
import os

In [2]:
# files = [f'waveforms/{f}' for f in os.listdir('waveforms') if 'HG8707' in f or 'HJ3311' in f]
# files = [f'waveforms/{f}' for f in os.listdir('waveforms') if 'clean' in f and 'diast' not in f and 'trended' not in f]

class epahClassifier():
    def __init__(self, waveforms=None, labels=None):
        self.waveforms = waveforms
        self.labels = labels
    
    def load_waveforms_from_files(self, files):
        # Load time series into DataFrame
        df_list = []
        for file in files:
            # Load ECG data from the downloaded MATLAB file
            mat = loadmat(file)
            file_id = file.split('.')[0]
            file_id = file_id.split('/')[-1]
            
            # For now, there are two types of files
            # - Files ending in '_000' only contain ONE waveform
            # - Files containing 'clean' may contain MULTIPLE waveforms
            if 'wave' in mat:
                # Note: Each file only contains ONE waveform!
                wave = mat['wave'][0][0][13][3]

                for i, x in enumerate(wave):
                    df_dict = {
                        'id':      file_id,
                        'time':    i,
                        'value':   wave[i],
                    }

                    df_list.append(df_dict)
                    
            elif 'savecleanforms' in mat:
                # Note: Each file may contain multiple waveforms!
                waves = mat['savecleanforms']['wave']
                
                # Create entries for each waveform in the mat file
                for wave_num, wave in enumerate(waves.item()[0]):
                    wave = wave[0]
                    
                    for i, x in enumerate(wave):
                        df_dict = {
                            'id':      f'{file_id}_{wave_num}',
                            'time':    i,
                            'value':   wave[i],
                        }
                        
                        df_list.append(df_dict)

        self.waveforms = pd.DataFrame(df_list)
    
    def load_labels_from_files(self, files):
        # Load labels into Data Series
        labels_dict = {}
        ind = []
        for file in files:
            file_id = file.split('_')[0]
            
            if mat.has_key('wave'):
                # Note: Each file only contains ONE waveform!
                labels_dict[file_id] = int(data[0] % 2)
                ind.append(file_id)
                
            elif mat.has_key('savecleanforms'):
                # Note: Each file may contain multiple waveforms!
                labels_dict[file_id] = int(data[0] % 2)
                ind.append(file_id)

        self.labels = pd.Series(labels_dict, index=ind)

In [73]:
epah = epahClassifier()

files_in_folder = [f'waveforms/{f}' for f in os.listdir('waveforms') if 'clean' in f and 'diast' not in f and 'trended' not in f and 'EL1030' not in f]
# files_in_folder = [f'waveforms/{f}' for f in os.listdir('waveforms') if 'clean' in f and'diast' not in f and 'trended' not in f]

df_labels = pd.read_csv('labels.csv')
df_labels['Filename'] = df_labels['Filename'].str.strip()
df_labels = df_labels.set_index(['Filename'])
df_labels['File_Exists'] = [any(filename.split('_')[0] in f for f in files_in_folder) for filename in df_labels.index]
df_labels['RHC_and_Exists'] = ~df_labels['RHC_or_CPET'] & df_labels['File_Exists']
df_labels['Label'] = df_labels['Source'].map({'control': 0, 'pah': 1, 'epah': 2})

files = [f'waveforms/{f.strip()}cleanforms.mat' for f in df_labels[df_labels['RHC_and_Exists']].index]

epah.load_waveforms_from_files(files)

print(f'Data Size: {len(files)}')

print(f'# Waveforms: {len(epah.waveforms.id.unique())}')

Data Size: 25
# Waveforms: 156


In [74]:
id_list = list(epah.waveforms['id'])

labels_dict = {}
for file_id in id_list:
    labels_dict[file_id] = df_labels.loc[file_id.split('clean')[0]]['Label']

epah.labels = pd.Series(labels_dict, index=list(labels_dict.keys()))

print(epah.labels.value_counts())

2    65
1    61
0    30
dtype: int64


In [66]:
waveforms = epah.waveforms['id'].unique()

use_interp = True
use_normalize = False
use_standardize = True

if use_interp:
    df_list = []
    for wave_id in waveforms:
        id_mask = epah.waveforms['id'].str.contains(wave_id, regex=False)
        wave = epah.waveforms[id_mask]['value']
        wave_interp = np.interp(np.linspace(0, len(wave), 500), np.arange(0, len(wave)), wave)

        for i, x in enumerate(wave_interp):
            df_dict = {
                'id':      wave_id,
                'time':    i,
                'value':   wave_interp[i],
            }

            df_list.append(df_dict)

    epah.waveforms = pd.DataFrame(df_list)

if use_normalize or use_standardize:
    for wave_id in waveforms:
        id_mask = epah.waveforms['id'].str.contains(wave_id, regex=False)
        if use_normalize:
            epah.waveforms.loc[id_mask, 'value_norm'] = (epah.waveforms[id_mask]['value'] - epah.waveforms[id_mask]['value'].min()) / (epah.waveforms[id_mask]['value'].max() - epah.waveforms[id_mask]['value'].min())
        elif use_standardize:
            epah.waveforms.loc[id_mask, 'value_norm'] = (epah.waveforms[id_mask]['value'] - epah.waveforms[id_mask]['value'].mean()) / epah.waveforms[id_mask]['value'].std()

df_waveforms = epah.waveforms.copy()
df_waveforms = df_waveforms.drop(columns=['value'])

### Create Pipeline & Model

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tsfresh.transformers import RelevantFeatureAugmenter
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
# Classes:
# 0: Control
# 1: PAH
# 2: EPAH

# Remove Class
# -1 -->  keep all 3 classes
# 0  -->  PAH vs EPAH
# 1  -->  Control vs EPAH
# 2  -->  Control vs PAH
class_remove = 2

y = epah.labels[epah.labels != class_remove]

X = pd.DataFrame(index=y.index)

# Split data into train and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# Ensure even split between labels
# FOR NOW, pick 2 patients randomly from each class
pt_list = list(set(ind.split('_')[0] for ind in y.index))

pt_labels = []
for pt in pt_list:
    label = y[[pt in ind for ind in y.index]][0]
    weight = sum([pt in ind for ind in y.index])
    
    pt_labels.append({
        'pt':     pt,
        'label':  label,
        'weight': weight,
    })

pt_df = pd.DataFrame(pt_labels)

if class_remove == 0:
    pt_test = pd.concat([pt_df[pt_df['label'] == 1].sample(n=3),
                         pt_df[pt_df['label'] == 2].sample(n=3)])
elif class_remove == 1:
    pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                         pt_df[pt_df['label'] == 2].sample(n=3)])
elif class_remove == 2:
    pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                         pt_df[pt_df['label'] == 1].sample(n=3),])
else:
    pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                         pt_df[pt_df['label'] == 1].sample(n=3),
                         pt_df[pt_df['label'] == 2].sample(n=3)])

display(pt_test)

y_train = y[[all(pt not in ind for pt in pt_test['pt']) for ind in y.index]]
X_train = X[[all(pt not in ind for pt in pt_test['pt']) for ind in X.index]]
y_test = y[[any(pt in ind for pt in pt_test['pt']) for ind in y.index]]
X_test = X[[any(pt in ind for pt in pt_test['pt']) for ind in X.index]]

print(f'Training Split: {len(y_train)}')
print(f'Testing  Split: {len(y_test)}')

In [ ]:
ppl = Pipeline([
        ('augmenter', RelevantFeatureAugmenter(column_id='id', column_sort='time')),
        ('classifier', RandomForestClassifier())
      ])

ppl.set_params(augmenter__timeseries_container=df_waveforms[df_waveforms['id'].isin(y.index)])

ppl.fit(X_train, y_train)

y_pred = ppl.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import plot_roc_curve

fig, ax = plt.subplots()
plot_roc_curve(ppl, X_test, y_test, ax=ax)
ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.6)
ax.legend()

### Calculate Relevant Features

In [43]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

In [ ]:
# Remove Class
# -1 -->  keep all 3 classes
# 0  -->  PAH vs EPAH
# 1  -->  Control vs EPAH
# 2  -->  Control vs PAH

classes = [0, 1, 2]

for class_remove in classes:
    y = epah.labels[epah.labels != class_remove]

    # Ensure even split between labels
    # FOR NOW, pick 2 patients randomly from each class
    pt_list = list(set(ind.split('_')[0] for ind in y.index))

    relevant_list = []
    relevant_dict = {}
    num_trials = 10
    for i in np.arange(0, num_trials):
        pt_labels = []
        for pt in pt_list:
            label = y[[pt in ind for ind in y.index]][0]
            weight = sum([pt in ind for ind in y.index])

            pt_labels.append({
                'pt':     pt,
                'label':  label,
                'weight': weight,
            })

        pt_df = pd.DataFrame(pt_labels)

        if class_remove == 0:
            pt_test = pd.concat([pt_df[pt_df['label'] == 1].sample(n=3),
                                 pt_df[pt_df['label'] == 2].sample(n=3)])
        elif class_remove == 1:
            pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                                 pt_df[pt_df['label'] == 2].sample(n=3)])
        elif class_remove == 2:
            pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                                 pt_df[pt_df['label'] == 1].sample(n=3),])
        else:
            pt_test = pd.concat([pt_df[pt_df['label'] == 0].sample(n=2),
                                 pt_df[pt_df['label'] == 1].sample(n=3),
                                 pt_df[pt_df['label'] == 2].sample(n=3)])

    #     display(pt_test)

        y_train = y[[all(pt not in ind for pt in pt_test['pt']) for ind in y.index]]
        # X_train = X[[all(pt not in ind for pt in pt_test['pt']) for ind in X.index]]
        y_test = y[[any(pt in ind for pt in pt_test['pt']) for ind in y.index]]
        # X_test = X[[any(pt in ind for pt in pt_test['pt']) for ind in X.index]]

    #     print(f'Training Split: {len(y_train)}')
    #     print(f'Testing  Split: {len(y_test)}')

        extracted_features = extract_features(epah.waveforms[epah.waveforms['id'].isin(y_train.index)], column_id='id', column_sort='time', column_value='value_norm')
        impute(extracted_features)
        df_relevance_table = calculate_relevance_table(extracted_features, y_train)

        for key, value in df_relevance_table[df_relevance_table['relevant']]['p_value'].items():
            relevant_dict.setdefault(key, []).append(value)

    with open(f'allRelevant_dict_{class_remove}_standardized.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(['Feature', 'Mean P Value', 'P Values', 'Num_Relevant(10)'])
        for key, value in sorted(list(relevant_dict.items())):
            writer.writerow([key, np.mean(value), value, len(value)])

    for key, value in list(relevant_dict.items()):
        if len(value) < num_trials:
            del relevant_dict[key]

    with open(f'relevant_dict_{class_remove}_standardized.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(['Feature', 'Mean P Value', 'P Values'])
        for key, value in sorted(list(relevant_dict.items())):
            writer.writerow([key, np.mean(value), value])

    sorted(list(relevant_dict.keys()))

In [67]:
# Calculate ALL features for ALL waveforms + Save to CSV
extracted_features = extract_features(epah.waveforms, column_id='id', column_sort='time', column_value='value')
impute(extracted_features)

extracted_features_labelled = extracted_features.copy()

for index, value in df_labels['Source'].items():
    extracted_features_labelled.loc[extracted_features_labelled.index.str.contains(index), 'label'] = value

extracted_features_labelled = extracted_features_labelled[['label'] + list(extracted_features.columns.values)]

extracted_features_labelled.to_csv('calculated_features.csv')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 26/26 [00:16<00:00,  1.57it/s]
C:\Users\andre\miniconda3\envs\epahClassification\lib\site-packages\tsfresh\utilities\dataframe_functions.py:171: RuntimeWarning: The columns ['value_norm__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


In [75]:
# Calculate ONLY RELEVANT features for ALL waveforms + Save to CSV
extracted_features = extract_features(epah.waveforms, column_id='id', column_sort='time', column_value='value')
impute(extracted_features)

# Remove Class
# -1 -->  keep all 3 classes
# 0  -->  PAH vs EPAH
# 1  -->  Control vs EPAH
# 2  -->  Control vs PAH

classes = [0, 1, 2]

for class_remove in classes:
    extracted_features_labelled = extracted_features.copy()

    for index, value in df_labels['Source'].items():
        extracted_features_labelled.loc[extracted_features_labelled.index.str.contains(index), 'label'] = value
        
    relevant_features = pd.read_csv(f'relevant_features/relevant_dict_{class_remove}.csv')
    
    extracted_features_labelled = extracted_features_labelled[['label'] + list(relevant_features['Feature'])]

    extracted_features_labelled.to_csv(f'calculated_relevantFeatures_{class_remove}.csv')

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 26/26 [00:07<00:00,  3.32it/s]
C:\Users\andre\miniconda3\envs\epahClassification\lib\site-packages\tsfresh\utilities\dataframe_functions.py:171: RuntimeWarning: The columns ['value__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


### Create Images of Each Waveform

In [29]:
# Plot of raw values
waves = epah.waveforms['id'].unique()

for wave in waves:
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    
    label = df_labels.loc[wave.split('clean')[0]]['Source']
    values = epah.waveforms[epah.waveforms['id'].str.contains(wave, regex=False)]['value_norm']
    
    ax.plot(np.arange(0, len(values)), values)
    ax.set_title(f'{wave}  •  {label.upper()}  •  Mean: {values.mean():0.3f} ± {values.std():0.3f}', fontsize=14)
    
#     break
    
    fig.savefig(f'waveforms_images/standardized/waveforms/{wave}_image.jpg')
    plt.close()

In [31]:
# Plot of CWT of waveforms
waves = epah.waveforms['id'].unique()

for wave in waves:
    fig, ax = plt.subplots(1, 1, figsize=(7, 5))
    
    label = df_labels.loc[wave.split('clean')[0]]['Source']
    values = epah.waveforms[epah.waveforms['id'].str.contains(wave, regex=False)]['value']
    
    num_steps = 256
    scales = np.arange(1, num_steps+1)
    wavelet_type = 'mexh'
    coefs, freqs = pywt.cwt(values, scales, wavelet_type)
    im = ax.imshow(coefs, cmap='coolwarm')
    cb = plt.colorbar(im, ax=ax)
    cb.set_label('Intensity', rotation=270)
    ax.set_xlabel('Time')
    ax.set_ylabel('Scale')
    ax.set_title(f'{wave}  •  {label.upper()}', fontsize=14)
    
    fig.tight_layout()
    
#     break
    
    fig.savefig(f'waveforms_images/standardized/cwt/{wavelet_type}/{wave}_image.jpg')
    plt.close()

### MISC

In [ ]:
# Edits a column in csv and resaves

from ast import literal_eval

filename = 'allRelevant_dict_0_standardized.csv'

df = pd.read_csv(filename)

df['P Values'] = df['P Values'].apply(literal_eval)
df['Num_Relevant(10)'] = [len(x) for x in df['P Values']]

df.to_csv(filename, index=False)